# Setting-up environment

In [ ]:
!cp -r ../input/updated-code-3-7/* /kaggle/working/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

In [ ]:
!python run_compile_all_cython.py

# Imports

In [ ]:
import time
import optuna
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count

In [ ]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix

In [ ]:
seed = 69
np.random.seed(seed)

# Data Import and Preprocessing

In [ ]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [ ]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

In [ ]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [ ]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

## Conversion to Sparse Matrix

In [ ]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [ ]:
urm_train, urm_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
#urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

# **Custom Models**

In [ ]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        
        item_weights_2 = item_weights_2 - np.array(item_weights_2).mean()
        print(item_weights_2.shape)
        item_weights_2[item_weights_2 < 0] = 0
        item_weights_2[item_weights_2 > 1] = 0.99

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [ ]:
class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

In [ ]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 300, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [ ]:
class GeneralizedLinearHybridRecommender(BaseRecommender):
    """
    This recommender merges N recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedLinearHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedLinearHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.recommenders = recommenders

    def fit(self, alphas=None):
        self.alphas = alphas

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.alphas[0]*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        for index in range(1,len(self.alphas)):
            result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [ ]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

In [ ]:
class RankHybridRecommender(BaseRecommender):
    """ RankHybridRecommender
    Hybrid of two Recommendations using weighted borda count

    """

    RECOMMENDER_NAME = "RankHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(RankHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha
        

    def recommend(self, user_id_array, cutoff = None, remove_seen_flag=True, items_to_compute = None, 
                  remove_top_pop_flag = False, remove_custom_items_flag = False, return_scores = False):
        recommendations_rec1, scores_batch_rec1 = self.recommender_1.recommend(user_id_array, cutoff, remove_seen_flag, items_to_compute, remove_top_pop_flag, remove_custom_items_flag, return_scores)
        recommendations_rec2, scores_batch_rec2 = self.recommender_2.recommend(user_id_array, cutoff, remove_seen_flag, items_to_compute, remove_top_pop_flag, remove_custom_items_flag, return_scores)
        #print(np.shape(recommendations_rec1))
        rankings = [recommendations_rec1, recommendations_rec2]
        #print(np.shape(rankings))
        weights = [self.alpha, 1-self.alpha] 
        
        items_to_compute = np.arange(self.n_items)
        
        final_recommendations = weighted_borda_count(np.array(rankings), weights)
        if return_scores:
            return final_recommendations, np.zeros(shape=(len(user_id_array), len(items_to_compute)))

        else:
            return final_recommendations
        

In [ ]:
def weighted_borda_count(rankings, weights):
    """
    Aggregate rankings using the weighted Borda count method for multiple recommenders and users,
    considering unique recommendations from each recommender.
    
    :param rankings: A 3D list or array of rankings, shape (n_recommenders, n_users, n_recommendations)
    :param weights: A list of weights corresponding to each recommender's importance.
    :return: An aggregated ranking list for each user.
    """
    if rankings.shape[0] != len(weights):
        raise ValueError("Number of recommenders in rankings must match the number of weights.")

    n_recommenders, n_users, n_recommendations = rankings.shape
    
    # Initialize a list to store the total Borda count for each item for each user
    borda_scores = [{} for _ in range(n_users)]

    # Loop through each recommender, user, and their rankings
    for recommender_index, weight in enumerate(weights):
        for user_index in range(n_users):
            # Get the maximum possible Borda count in current recommender's list
            max_borda_count = n_recommendations * weight
            for rank_index, item_id in enumerate(rankings[recommender_index, user_index]):
                # The Borda count for the current rank and weight
                points = (max_borda_count - rank_index * weight)
                if item_id in borda_scores[user_index]:
                    borda_scores[user_index][item_id] += points
                else:
                    borda_scores[user_index][item_id] = points

    # Sort the items for each user by their Borda count in descending order
    aggregated_rankings = []
    for user_index in range(n_users):
        user_ranking = sorted(borda_scores[user_index].items(), key=lambda x: x[1], reverse=True)
        # Extract the item IDs from the sorted tuples
        aggregated_rankings.append([item_id for item_id, _ in user_ranking])

    # Return the aggregated rankings for each user
    return aggregated_rankings

# **Best Model Params**

In [ ]:
ItemKNN_params = {
 'topK': 21,
 'shrink': 1462,
 'similarity': 'tanimoto',
 'normalize': False,
 'feature_weighting': 'BM25'
}

In [ ]:
RP3Beta_params = {
 'alpha': 0.3302958327908062,
 'beta': 0.14271386569051958,
 'topK': 29,
 'normalize_similarity': True
}

In [ ]:
alpha_itemknn_rp3beta=0.7381515719042592

In [ ]:
alpha_slim_rp3beta = 0.5356582511094522

In [ ]:
UserKNN_params = {
 'topK': 387,
 'shrink': 1,
 'similarity': 'cosine',
 'normalize': True,
 'feature_weighting': 'TF-IDF'
}

In [ ]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

In [ ]:
SLIM_params = {
 'l1_ratio': 0.04324773367371399,
 'alpha': 0.001220701931267383,
 'topK': 971
}

In [ ]:
NMF_params = {
    'l1_ratio': 0.005734775635120469,
    'num_factors': 134,
    'beta_loss': 'frobenius',
    'init_type': 'nndsvda',
    'solver': 'multiplicative_update'
}

In [ ]:
IALS_params = {
'factors': 216, 
'confidence_scaling': 'linear', 
'alpha': 2.971319492415353, 
'epsilon': 0.27505837539726546, 
'regularization': 0.0062196937352773235
}

## Pairwise tuning

In [ ]:
itemKNNCF = ItemKNNCFRecommender(urm_train)
itemKNNCF.fit(**ItemKNN_params)

In [ ]:
rp3beta = RP3betaRecommender(urm_train)
rp3beta.fit(**RP3Beta_params)

In [ ]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
slim.fit(**SLIM_params, workers = int(cpu_count()))

### Plotting scores

In [ ]:
import seaborn as sns
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [ ]:
user_id = 0

rp3beta_scores = rp3beta._compute_item_score(user_id).flatten()
slim_scores = slim._compute_item_score(user_id).flatten()
itemknn_scores = itemKNNCF._compute_item_score(user_id).flatten()  

In [ ]:
_ = plt.figure(figsize=(16,7))
ax = sns.scatterplot(data=rp3beta_scores, alpha=0.5, color = "red")
ax = sns.scatterplot(data=slim_scores, alpha=0.5, color = "blue")
#ax = sns.scatterplot(data=itemknn_scores, alpha=0.5, color = "green")
ax.legend(['rp3beta', 'slim'])

### Instantiate Hybrid

In [ ]:
new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
    
slim_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
slim_rp3beta.fit(new_similarity)

### Tuning Hybrid

In [ ]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [ ]:
def objective_function_slim_rp3beta(optuna_trial):
    alpha_slim_rp3beta = optuna_trial.suggest_float("alpha_slim_rp3beta", 0.50, 0.67)
    new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
    recommender_instance = ItemKNNCustomSimilarityRecommender(urm_train)
    recommender_instance.fit(new_similarity)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_slim_rp3beta = optuna.create_study(direction="maximize")

save_results_slim_rp3beta = SaveResults()

optuna_study_slim_rp3beta.optimize(objective_function_slim_rp3beta,
                      callbacks=[save_results_slim_rp3beta],
                      n_trials = 20)

In [ ]:
#alpha_slim_rp3beta = 0.5815552360509464
#alpha_slim_rp3beta = 0.522113314123003
alpha_slim_rp3beta = 0.5356582511094522

In [ ]:
pruned_trials = [t for t in optuna_study_slim_rp3beta.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_slim_rp3beta.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_slim_rp3beta.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_slim_rp3beta.best_trial.value)

print("Best params:")
print(optuna_study_slim_rp3beta.best_trial.params)

In [ ]:
'''def objective_function_hybrid_item_rp3beta(optuna_trial):
    alpha = optuna_trial.suggest_float("alpha", 0, 1)
    
    recommender_instance = ScoresHybridRecommender(urm_train, itemKNN_rp3beta, slim)
    recommender_instance.fit(alpha)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]'''

In [ ]:
slim_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
slim_rp3beta.fit(new_similarity, )

In [ ]:
result_df, _ = evaluator_test.evaluateRecommender(slim_rp3beta)

In [ ]:
result_df

In [ ]:
result_df, _ = evaluator_validation.evaluateRecommender(slim_rp3beta)
result_df

# Adding iALS

In [ ]:
ials = FastIALSRecommender(urm_train)
ials.fit(**IALS_params)

In [ ]:
def objective_function_hybrid(optuna_trial):
    alpha = optuna_trial.suggest_float("alpha", 0, 1)
    
    recommender_instance = ScoresHybridRecommender(urm_train, slim_rp3beta, ials)
    recommender_instance.fit(alpha)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_hybrid = optuna.create_study(direction="maximize")

save_results_hybrid = SaveResults()

optuna_study_hybrid.optimize(objective_function_hybrid,
                      callbacks=[save_results_hybrid],
                      n_trials = 300)

## Submission

In [ ]:
urm_full = urm_train+urm_validation

In [ ]:
top_pop_final = TopPop(urm_full)
top_pop_final.fit()

In [ ]:
rp3beta = RP3betaRecommender(urm_full)
rp3beta.fit(**RP3Beta_params)

In [ ]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_full)
slim.fit(**SLIM_params, workers = int(cpu_count()))

In [ ]:
slim_rp3beta = ItemKNNCustomSimilarityRecommender(urm_full)
new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
slim_rp3beta.fit(new_similarity)

In [ ]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix, recommender: BaseRecommender, recommendation_length):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

    submission = []

    rec_users_arr = users_ids_and_mappings.mapped_user_id.to_numpy()
    recommendations = recommender.recommend(user_id_array= rec_users_arr, cutoff=recommendation_length)

    i = 0

    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations[i]]))
        i+=1

    return submission

In [ ]:
users_to_recommend = target_dataframe.to_numpy().flatten()
users_to_recommend.shape

## Mappings

In [ ]:
mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

In [ ]:
urm_ids = np.unique(ratings.user_id)
missing_users = set([i for i in users_to_recommend]) - set([i for i in urm_ids])
missing_users = np.array([x for x in missing_users])

In [ ]:
!mkdir /kaggle/working/Results

In [ ]:
def write_submission(submissions, name):
    with open(f"/kaggle/working/Results/{name}.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
        f.close()

In [ ]:
submission = prepare_submission(ratings, users_to_recommend, urm_full, slim_rp3beta, recommendation_length=10)

In [ ]:
rec_missing = top_pop_final.recommend(missing_users, cutoff=10, remove_seen_flag=False)

In [ ]:
for user_id in missing_users:
    submission.append((user_id, [mapping_to_item_id[item_id] for item_id in rec_missing[0]]))

In [ ]:
(len(submission), len(users_to_recommend))

In [ ]:
write_submission(submission, "slim_rp3beta")